In [17]:
import numpy as np
import sys
import re
import os
import pandas as pd
import warnings
from pathlib import Path
from typing import List, Dict, Tuple, Any, Union, Callable
from tqdm import tqdm
import csv
import json
from dataclasses import dataclass
from configobj import ConfigObj

In [3]:
WDIR = Path().cwd().parent

PLANET_DB_PATH = str(WDIR / "data/planet_database_composite.csv")
OPACITY_PATH = str(WDIR / "data/Input/xsec/xsec_sampled_R15000_0.3-15")
CIA_PATH = str(WDIR / "data/Input/cia/hitran")
MOLECULE_PATH = str(WDIR / "data/molecule_db.json")

In [5]:
def read_json_file(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

print(read_json_file(MOLECULE_PATH))

[{'molecule': 'H2O', 'abundance': -3, 'type': 'ConstantGas'}, {'molecule': 'CH4', 'abundance': -7, 'type': 'ConstantGas'}, {'molecule': 'NH3', 'abundance': -7, 'type': 'ConstantGas'}, {'molecule': 'CO', 'abundance': -7, 'type': 'ConstantGas'}, {'molecule': 'CO2', 'abundance': -7, 'type': 'ConstantGas'}, {'molecule': 'HCN', 'abundance': -7, 'type': 'ConstantGas'}, {'molecule': 'Na', 'abundance': -7, 'type': 'ConstantGas'}, {'molecule': 'K', 'abundance': -7, 'type': 'ConstantGas'}, {'molecule': 'TiO', 'abundance': -15, 'type': 'ConstantGas'}, {'molecule': 'VO', 'abundance': -15, 'type': 'ConstantGas'}, {'molecule': 'FeH', 'abundance': -15, 'type': 'ConstantGas'}, {'molecule': 'e-', 'abundance': -15, 'type': 'ConstantGas'}]


In [19]:
config = ConfigObj(str(WDIR / "data/default.par"))

config



ConfigObj({'Global': {'xsec_path': '/path/to/xsec', 'cia_path': '/path/to/cia'}, 'Chemistry': {'chemistry_type': 'taurex', 'fill_gases': ['H2', 'He'], 'ratio': '4.8962e-2', 'CO2': {'gas_type': 'constant', 'mix_ratio': '1.1185e-4'}, 'N2': {'gas_type': 'constant', 'mix_ratio': '3.00739e-9'}}, 'Temperature': {'profile_type': 'isothermal', 'T': '1265.97764826'}, 'Pressure': {'profile_type': 'Simple', 'atm_min_pressure': '1e-0', 'atm_max_pressure': '1e6', 'nlayers': '30'}, 'Planet': {'planet_type': 'Simple', 'planet_mass': '1.0', 'planet_radius': '0.999430455828'}, 'Star': {'star_type': 'blackbody', 'temperature': '5800'}, 'Model': {'model_type': 'transmission', 'Absorption': {}, 'CIA': {'cia_pairs': ['H2-He', 'H2-H2']}, 'Rayleigh': {}}, 'Observation': {'observation': 'spectra_w_offsets', 'path_spectra': ['spectrum1.txt', 'spectrum2.txt', 'spectrum3.txt'], 'offsets': ['0.0', '0.0', '0.0'], 'slopes': ['0.0', '0.0', '0.0']}, 'Binning': {}, 'Instrument': {}, 'Optimizer': {'optimizer': 'nestle'

In [36]:
def write_global(config, settings=None):
    config['Global']['xsec_path'] = str(OPACITY_PATH)
    config['Global']['cia_path'] = str(CIA_PATH)

def write_chemistry(config, settings=None):
    config['Chemistry']['chemistry_type'] = 'taurex'
    config['Chemistry']['fill_gases'] = ['H2', 'He']
    config['Chemistry']['ratio'] = 0.17

def _write_abstract(config, settings, level=0):
    for k, v in settings.items():
        config[k] = {}
        try:
            assert isinstance(v, dict)
            _write_abstract(config=config[k], settings=v, level=level+1)
        except AssertionError:
            config[k] = v
            return

config_empt = ConfigObj()

def make_Obs_ObsFit_dict(path_list, fitting_bounds=None, fitting_mode="linear"):
    observation_dict = {
        'observation': 'spectra_w_offsets',
        'path_spectra': ['spectrum1.txt',
                         'spectrum2.txt',
                         'spectrum3.txt'],
        'offsets': ["0.0"] * len(path_list),
        'slopes': ["0.0"] * len(path_list),
    }

    if fitting_mode is None:
        fitting_mode = ['-1e-15', '1e-15']

    if isinstance(fitting_bounds[0], (str, float)) and isinstance(fitting_mode, str):
        fitting_dict = {}
        for i, __ in enumerate(path_list):

            obs_fit_dict = {
                f'Offset_{i+1}:fit': 'True',
                f'Offset_{i+1}:bounds': fitting_mode,
                f'Offset_{i+1}:mode': fitting_mode,
            }

            fitting_dict = {**fitting_dict, **obs_fit_dict}
    else:
        raise NotImplementedError("Only same options for each observation fit is supported.")

    out_dict = {
        'Observation': observation_dict,
        'Fitting': fitting_dict,
    }
    return out_dict



def make_Chem_dict(molecule_dp_path, which_molecules=None, settings=None):
    # TODO: use types as type hinting?
    types = {
        'ConstantGas': ConstantGas,
        'TwoLayerGas': TwoLayerGas,
        'HydrogenIon': HydrogenIon,
    }

    # TODO: unpack, into which gasses, k, v
    gas_para_db = read_json_file(MOLECULE_PATH)

    gases_dict = {}



    # TODO: add gas_type as attr to json separate from type? for .par files
    for gas in which_molecules:
        gas_dict ={gas["molecule"]:
            {
                "gas_type": gas["gas_type"],
                "mix_ratio": 10 ** gas["abundance"],
            }
        }
        gases_dict = {**gases_dict, **gas_dict}

    chemistry_dict = {
        "chemistry_type": "free",  # TODO: free or taurex?
        "fill_gasses": ["H2", "He"],
        "ratio": 0.17,
        **gases_dict,
    }

    return chemistry_dict

def make_Temp_dict(target, settings=None):
    if settings is None:
        settings = {}

    # TODO: default is isothermal. Check re npoint or Guillot2016/2018?
    default_dict = {
        "profile_type": "isothermal",
        "T": target["Equilibrium Temperature [K]"],
    }

    # TODO: issues because errors are thrown if unexpected keys in Temperature config
    # overwrite partial if type key is same, overwrite fully if mismatch
    return {"Temperature": {**default_dict, **settings}}


def make_Press_dict(settings=None):
    if settings is None:
        settings = {}

    return {"Pressure": settings}


def make_Planet_dict(target, settings=None):

    # planet_radius=target["Planet Radius [Jupiter Radius]"]
    # planet_mass=target["Planet Mass or Mass*sin(i) [Jupiter Mass]"]


    planet_dict = {
        "planet_type": "Simple" ##
        # ... keys?
    }

    return {"Planet": settings}


def make_bounds_from_derived_para(para, interval=None, mode=None, fit=True):
    if interval is None:
        interval = np.array([0.1, 10.])
    if mode is None:
        mode = "linear" # TODO: check with docs if custom priors instead of bounds

    out = {
        f"{para['name']}:fit": fit,
        f"{para['name']}:bounds": list(interval * para["value"]),
        f"{para['name']}:mode": mode,
    }

    return {"Fitting": out}








dic = {'Global': {'xsec_path': '/path/to/xsec', 'cia_path': '/path/to/cia'}, 'Chemistry': {'chemistry_type': 'taurex', 'fill_gases': ['H2', 'He'], 'ratio': '4.8962e-2', 'CO2': {'gas_type': 'constant', 'mix_ratio': '1.1185e-4'}, 'N2': {'gas_type': 'constant', 'mix_ratio': '3.00739e-9'}}, 'Temperature': {'profile_type': 'isothermal', 'T': '1265.97764826'}, 'Pressure': {'profile_type': 'Simple', 'atm_min_pressure': '1e-0', 'atm_max_pressure': '1e6', 'nlayers': '30'}, 'Planet': {'planet_type': 'Simple', 'planet_mass': '1.0', 'planet_radius': '0.999430455828'}, 'Star': {'star_type': 'blackbody', 'temperature': '5800'}, 'Model': {'model_type': 'transmission', 'Absorption': {}, 'CIA': {'cia_pairs': ['H2-He', 'H2-H2']}, 'Rayleigh': {}}, 'Observation': {'observation': 'spectra_w_offsets', 'path_spectra': ['spectrum1.txt', 'spectrum2.txt', 'spectrum3.txt'], 'offsets': ['0.0', '0.0', '0.0'], 'slopes': ['0.0', '0.0', '0.0']}, 'Binning': {}, 'Instrument': {}, 'Optimizer': {'optimizer': 'nestle', 'num_live_points': '100', 'method': 'single'}, 'Fitting': {'Offset_1:fit': 'True', 'Offset_1:bounds': ['-1e-15', '1e-15'], 'Offset_1:mode': 'linear', 'Offset_2:fit': 'True', 'Offset_2:bounds': ['-1e-15', '1e-15'], 'Offset_2:mode': 'linear', 'Offset_3:fit': 'True', 'Offset_3:bounds': ['-1e-15', '1e-15'], 'Offset_3:mode': 'linear', 'Slope_1:fit': 'True', 'Slope_1:bounds': ['-1e-15', '1e-15'], 'Slope_1:mode': 'linear', 'Slope_2:fit': 'True', 'Slope_2:bounds': ['-1e-15', '1e-15'], 'Slope_2:mode': 'linear', 'Slope_3:fit': 'True', 'Slope_3:bounds': ['-1e-15', '1e-15'], 'Slope_3:mode': 'linear'}}

# _write_abstract(config=config_empt, settings=dic)

config_empt = ConfigObj(dic)

config_empt.filename = str(WDIR / "data/test.par")

config_empt.write()

print(config_empt)

# write_global(config)
# write_chemistry(config)
#
# config.write()

{'Global': {'xsec_path': '/path/to/xsec', 'cia_path': '/path/to/cia'}, 'Chemistry': {'chemistry_type': 'taurex', 'fill_gases': ['H2', 'He'], 'ratio': '4.8962e-2', 'CO2': {'gas_type': 'constant', 'mix_ratio': '1.1185e-4'}, 'N2': {'gas_type': 'constant', 'mix_ratio': '3.00739e-9'}}, 'Temperature': {'profile_type': 'isothermal', 'T': '1265.97764826'}, 'Pressure': {'profile_type': 'Simple', 'atm_min_pressure': '1e-0', 'atm_max_pressure': '1e6', 'nlayers': '30'}, 'Planet': {'planet_type': 'Simple', 'planet_mass': '1.0', 'planet_radius': '0.999430455828'}, 'Star': {'star_type': 'blackbody', 'temperature': '5800'}, 'Model': {'model_type': 'transmission', 'Absorption': {}, 'CIA': {'cia_pairs': ['H2-He', 'H2-H2']}, 'Rayleigh': {}}, 'Observation': {'observation': 'spectra_w_offsets', 'path_spectra': ['spectrum1.txt', 'spectrum2.txt', 'spectrum3.txt'], 'offsets': ['0.0', '0.0', '0.0'], 'slopes': ['0.0', '0.0', '0.0']}, 'Binning': {}, 'Instrument': {}, 'Optimizer': {'optimizer': 'nestle', 'num_liv